## Part 2. Database Engineering - Creating queries

* Use Pandas to read your cleaned measurements and stations CSV data.

* Using SQLAlchemy to model table schemas.

* Creating a sqlite database for tables.

### Reading clean data

In [1]:
# Importing dependecies
import pandas as pd

In [2]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float

In [3]:
# Creating path to csv files
measurements = "Resources/clean_hawaii_measurements.csv"
stations = "Resources/clean_hawaii_stations.csv"

In [4]:
# Reading csv file with measurements data
measures = pd.read_csv(measurements)
measures.head(10)

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-07,0.06,70
5,USC00519397,2010-01-08,0.00,64
6,USC00519397,2010-01-09,0.00,68
7,USC00519397,2010-01-10,0.00,73
8,USC00519397,2010-01-11,0.01,64
9,USC00519397,2010-01-12,0.00,61


In [5]:
# Reading csv file with stations data
stations = pd.read_csv(stations)
stations.head()

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


### Creating sqlite database

In [6]:
# Create an engine to a SQLite database file
engine = create_engine("sqlite:///./Resources/hawaii.sqlite")

In [7]:
# Create a connection to the engine called `conn`
conn = engine.connect()

In [8]:
# Creating tables in sqlite
Base = declarative_base()

class Measurements(Base):
    __tablename__ = 'measures'

    id = Column(Integer, primary_key=True)
    station = Column(Text)
    date = Column(Text)
    prcp = Column(Float)
    tobs = Column(Integer)
    
class Stations(Base):
    __tablename__ = 'stations'

    id = Column(Integer, primary_key=True)
    station = Column(Text)
    name = Column(Text)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
   
    def __repr__(self):
        return f"id={self.id}, name={self.name}"

In [9]:
# Use `create_all` to create the table in the database
Base.metadata.create_all(engine)

In [10]:
# Converting tables to disctionaries
measures_dict = measures.to_dict(orient='records')
stations_dict = stations.to_dict(orient='records')

In [11]:
print(measures_dict[:5])

[{'station': 'USC00519397', 'date': '2010-01-01', 'prcp': 0.08, 'tobs': 65}, {'station': 'USC00519397', 'date': '2010-01-02', 'prcp': 0.0, 'tobs': 63}, {'station': 'USC00519397', 'date': '2010-01-03', 'prcp': 0.0, 'tobs': 74}, {'station': 'USC00519397', 'date': '2010-01-04', 'prcp': 0.0, 'tobs': 76}, {'station': 'USC00519397', 'date': '2010-01-07', 'prcp': 0.06, 'tobs': 70}]


In [12]:
print(stations_dict[:5])

[{'station': 'USC00519397', 'name': 'WAIKIKI 717.2, HI US', 'latitude': 21.2716, 'longitude': -157.8168, 'elevation': 3.0}, {'station': 'USC00513117', 'name': 'KANEOHE 838.1, HI US', 'latitude': 21.4234, 'longitude': -157.8015, 'elevation': 14.6}, {'station': 'USC00514830', 'name': 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 'latitude': 21.5213, 'longitude': -157.8374, 'elevation': 7.0}, {'station': 'USC00517948', 'name': 'PEARL CITY, HI US', 'latitude': 21.3934, 'longitude': -157.9751, 'elevation': 11.9}, {'station': 'USC00518838', 'name': 'UPPER WAHIAWA 874.3, HI US', 'latitude': 21.4992, 'longitude': -158.0111, 'elevation': 306.6}]


In [13]:
# Using MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [14]:
# Saving the reference to the `measures` table as a variable called `table`
measures_table = sqlalchemy.Table('measures', metadata, autoload=True)
stations_table = sqlalchemy.Table('stations', metadata, autoload=True)

In [15]:
# Removing any pre-existing data.
conn.execute(measures_table.delete())

In [16]:
# Removing any pre-existing data.
conn.execute(stations_table.delete())

In [17]:
# Use `table.insert()` to insert the data into the table
# The SQL table is populated during this step
conn.execute(measures_table.insert(), measures_dict)

In [18]:
# Use `table.insert()` to insert the data into the table
# The SQL table is populated during this step
conn.execute(stations_table.insert(), stations_dict)

In [19]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from measures limit 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65),
 (2, 'USC00519397', '2010-01-02', 0.0, 63),
 (3, 'USC00519397', '2010-01-03', 0.0, 74),
 (4, 'USC00519397', '2010-01-04', 0.0, 76),
 (5, 'USC00519397', '2010-01-07', 0.06, 70)]

In [20]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from stations limit 5").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]